In [10]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import numpy as np
import os

from sklearn.metrics import r2_score

from NMR_model import DynamicNMRRegressor, DynamicNMRDataset
from NMR_preproc_methods import parse_directory, parse_data_file, splitSamples, split_data

In [11]:
# Парсим каждый файл и добавляем образцы в общий массив

file_path = "./kozlopan_ml100noise.txt" # ФАЙЛ ОБУЧАЮЩЕЙ ВЫБОРКИ
parsed_data = parse_data_file(file_path)

# ПРОВЕРКА
for i, sample in enumerate(parsed_data[:2]):
    print(f"Sample {i}:")
    print(f"Yi: {sample.get('Yi', [])}")
    for key in sorted(sample.keys()):
        if key.startswith("X["):
            print(f"{key}: first 5 values - {sample[key][:5]}")
    print()

# print(parsed_data[0]["X[0]"])
print(len(parsed_data)) # кол-во сигналов
print(len(parsed_data[0]["Yi"])) # кол-во целевых
print(len(parsed_data[0]["X[0]"])) # длина х0
print(len(parsed_data[0]["X[1]"])) # длина х1
print(len(parsed_data[0])-1) # кол-во х

Sample 0:
Yi: [0.0, 0.0]
X[0]: first 5 values - [0.99567765, 1.021505237, 0.9993298054, 1.011017203, 0.9911623001]
X[1]: first 5 values - [0.01429168601, 0.2336208224, 0.3934727311, 0.5301313996, 0.6267670989]

Sample 1:
Yi: [1.0, 0.25]
X[0]: first 5 values - [0.974399507, 0.9852434397, 0.9664190412, 0.9656493664, 1.004621387]
X[1]: first 5 values - [-0.009456628002, 0.1018851399, 0.1474274993, 0.2108162493, 0.2597899437]

100
2
400
60
2


Для теста парсим набор файлов, сгенерированных с шумом или без

In [12]:
parsed_data_for_test = parse_directory(directory_path="./SygnalsWithoutNoise") # Для тестов
print(len(parsed_data_for_test)) # кол-во сигналов в тестовой выборке

16


Но лучше иметь больше сигналов для теста, поэтому просто разделим большую обучающую базу на train/test

In [13]:
train_data, test_data = split_data(parsed_data, train_ratio=0.8, shuffle=True, random_seed=42)

print(f"Всего образцов: {len(parsed_data)}")
print(f"Обучающая выборка: {len(train_data)} образцов")
print(f"Тестовая выборка: {len(test_data)} образцов")

Всего образцов: 100
Обучающая выборка: 80 образцов
Тестовая выборка: 20 образцов


In [14]:
# x_train, y_train = splitSamples(parsed_data)
# x_test, y_test = splitSamples(parsed_data_for_test)
x_train, y_train = splitSamples(train_data)
x_test, y_test = splitSamples(test_data)

DATASET & MODEL

In [15]:
batch_size = 32
train_dataset = DynamicNMRDataset(*x_train, y=y_train)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = DynamicNMRDataset(*x_test, y=y_test)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Гибкость: Модель адаптируется к любому количеству экспериментов (M) и целей (N).

Масштабируемость: Можно добавлять новые типы сигналов без изменения кода.

Автоматическая адаптация: Размеры слоев вычисляются динамически.


In [16]:
input_dims = [len(x[0]) for x in x_train]  # Длины каждого X[i]
num_targets = len(y_train[0])  # Количество целевых переменных (Yi)

assert input_dims == [len(x[0]) for x in x_test] and num_targets == len(y_test[0]), "Несоответствие размеров train/test"
print(input_dims, num_targets)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Используемое устройство:", device)

model = DynamicNMRRegressor(input_dims, num_targets)
model.to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)

[400, 60] 2
Используемое устройство: cpu


Цикл обучения

In [17]:
for epoch in range(20):
    model.train()
    train_running_loss = 0.0
    
    for batch in train_dataloader:
        *x_batch, y_batch = batch
        x_batch = [x.to(device) for x in x_batch]
        y_batch = y_batch.to(device)
        
        optimizer.zero_grad()
        outputs = model(*x_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        
        train_running_loss += loss.item()
    
    # =========

    model.eval()
    test_running_loss = 0.0
    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        for batch in test_dataloader:
            *x_batch, y_batch = batch
            x_batch = [x.to(device) for x in x_batch]
            y_batch = y_batch.to(device)
            
            outputs = model(*x_batch)
            loss = criterion(outputs, y_batch)
            test_running_loss += loss.item()
            
            all_preds.append(outputs.cpu().numpy())
            all_targets.append(y_batch.cpu().numpy())
    
    # предсказания и цели
    all_preds = np.concatenate(all_preds, axis=0)
    all_targets = np.concatenate(all_targets, axis=0)
    
    # метрики
    train_loss = train_running_loss / len(train_dataloader)
    test_loss = test_running_loss / len(test_dataloader)
    r2 = r2_score(all_targets, all_preds)
    
    print(f"Epoch {epoch + 1}")
    print(f"Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f} | R² Score: {r2:.4f}")
    print("-" * 50)

Epoch 1
Train Loss: 0.1005 | Test Loss: 0.0950 | R² Score: -1.3148
--------------------------------------------------
Epoch 2
Train Loss: 0.0789 | Test Loss: 0.0460 | R² Score: -0.6403
--------------------------------------------------
Epoch 3
Train Loss: 0.0525 | Test Loss: 0.0312 | R² Score: -0.2187
--------------------------------------------------
Epoch 4
Train Loss: 0.0574 | Test Loss: 0.0293 | R² Score: -0.3459
--------------------------------------------------
Epoch 5
Train Loss: 0.0453 | Test Loss: 0.0261 | R² Score: 0.0498
--------------------------------------------------
Epoch 6
Train Loss: 0.0381 | Test Loss: 0.0322 | R² Score: 0.0132
--------------------------------------------------
Epoch 7
Train Loss: 0.0360 | Test Loss: 0.0208 | R² Score: 0.1964
--------------------------------------------------
Epoch 8
Train Loss: 0.0237 | Test Loss: 0.0122 | R² Score: 0.2893
--------------------------------------------------
Epoch 9
Train Loss: 0.0228 | Test Loss: 0.0102 | R² Score: 0

SAVE MODEL STATE TO FILE

In [18]:
# torch.save(model, 'model.pth')
torch.save(model.state_dict(), 'model_weights_100noise.pth')



Потенциальные проблемы

=====

Производительность:

Каждая ветвь обрабатывается независимо → может увеличиться время обучения.

Решение: Использовать nn.ModuleDict для ветвей, если M фиксировано.

Нормализация данных:

Разные типы сигналов могут требовать разной предобработки.

Решение: Добавить параметр scalers в модель.

Интерпретируемость:

Сложнее отслеживать вклад каждого типа сигнала.

Решение: Визуализировать attention-веса или градиенты.